## Prediccción de valor de stock

Importación de librerias

In [61]:
import pandas as pd
import numpy as np


In [62]:
#Buscando e inicializando la instalación de Spark
import findspark
findspark.init()
findspark.find()


'c:\\Users\\HP\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\pyspark'

In [63]:
#Verificar la funcionalidad de Pyspark 
from pyspark.sql import SparkSession

Creación de sesión spark

In [64]:
spark_session = SparkSession.builder.appName('Reto_movilidad').getOrCreate()
spark_session

Lectura de datos

Este dataset consiste en el conjunto de datos econometricos mensuales de 2000 empresas entre el 2000 y 2022. Todos estos datos fuerón obtenidos de https://finance.yahoo.com/

In [93]:
df_spark_col  = spark_session.read.option('header', 'true').csv('econometrics2000_2022.csv')
df_spark_col.show()

+---+----------+------------------+------------------+------------------+------------------+---------+------------------+----+------------------+--------------------+
|_c0|      Date|              High|               Low|              Open|             Close|   Volume|         Adj Close|Name|                R9|                 r10|
+---+----------+------------------+------------------+------------------+------------------+---------+------------------+----+------------------+--------------------+
|  0|2000-01-03|18.007015228271484|17.462066650390625|17.912240982055664|17.959627151489258|2062807.0| 9.394301414489746| TSM|              null|                null|
|  1|2000-01-04| 18.67043113708496|17.912240982055664|17.959627151489258|18.078094482421875|2557142.0|  9.45626735687256| TSM|1.0065963134943363|0.006574461103341811|
|  2|2000-01-05|18.362417221069336|18.078094482421875| 18.24394989013672|18.196563720703125|2295467.0| 9.518234252929688| TSM|1.0065531927823723|0.006531619921748444

In [94]:
df_spark_col.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- R9: string (nullable = true)
 |-- r10: string (nullable = true)



Se puede observar que hay datos númericos que fueron leídos como texto, esto se tomará en cuenta para su futura modificación en aquellos que se vayan a utilizar

Eliminación de datos nulos

Se eliminan aquellos datos nulos debido a que los datos nulos no permiten entrenar al modelo

In [95]:
df_spark_col = df_spark_col.na.drop()

In [96]:
df_spark_col.show()

+---+----------+------------------+------------------+------------------+------------------+---------+------------------+----+------------------+--------------------+
|_c0|      Date|              High|               Low|              Open|             Close|   Volume|         Adj Close|Name|                R9|                 r10|
+---+----------+------------------+------------------+------------------+------------------+---------+------------------+----+------------------+--------------------+
|  1|2000-01-04| 18.67043113708496|17.912240982055664|17.959627151489258|18.078094482421875|2557142.0|  9.45626735687256| TSM|1.0065963134943363|0.006574461103341811|
|  2|2000-01-05|18.362417221069336|18.078094482421875| 18.24394989013672|18.196563720703125|2295467.0| 9.518234252929688| TSM|1.0065531927823723|0.006531619921748444|
|  3|2000-01-06|17.793773651123047|17.059276580810547| 17.77008056640625|17.438371658325195|1539456.0| 9.121644020080566| TSM|0.9583332285141674| -0.0425593007666305

Eliminación de columnas que no se ocuparán en el modelo

Las columnas que se eliminarán se hace con el motivo de que en el contexto son practicamente los mismos valores que se quieren llegar a predecir

In [97]:
df_spark_col = df_spark_col.drop('_c0', 'High', 'Low', 'Close', 'r10', 'R9', 'Date')
df_spark_col.show()

+------------------+---------+------------------+----+
|              Open|   Volume|         Adj Close|Name|
+------------------+---------+------------------+----+
|17.959627151489258|2557142.0|  9.45626735687256| TSM|
| 18.24394989013672|2295467.0| 9.518234252929688| TSM|
| 17.77008056640625|1539456.0| 9.121644020080566| TSM|
| 18.05440139770508|1511230.0| 9.468661308288574| TSM|
| 18.69412422180176|1102626.0| 9.852862358093262| TSM|
| 18.81259155273437|2573761.0| 9.716533660888672| TSM|
|19.002140045166016|1407826.0|  9.87764835357666| TSM|
|18.954753875732425|1751803.0| 10.07594394683838| TSM|
| 19.52339553833008|4445851.0|10.943493843078612| TSM|
| 20.04465103149414|2009786.0|10.670833587646484| TSM|
|20.400053024291992|1653939.0|10.596468925476074| TSM|
| 20.68437385559082|3375671.0|10.435358047485352| TSM|
| 20.94500160217285|2109234.0|10.831949234008787| TSM|
|21.371484756469727|3403896.0|11.835826873779297| TSM|
|22.627235412597656|3588546.0| 12.98842716217041| TSM|
|25.494142

In [99]:
df_spark_col = df_spark_col.withColumnRenamed(
  "Adj Close", "adjClose")

Transformar tipo de dato, debido a que es tipo string y se ocupa en tipo double

In [100]:

df_spark_col = df_spark_col.withColumn("Volume",df_spark_col.Volume.cast('double'))
df_spark_col = df_spark_col.withColumn("Open",df_spark_col.Open.cast('double'))
df_spark_col = df_spark_col.withColumn("adjClose",df_spark_col.adjClose.cast('double'))



In [101]:
from pyspark.ml.feature import VectorAssembler

In [102]:
df_spark_col.printSchema()

root
 |-- Open: double (nullable = true)
 |-- Volume: double (nullable = true)
 |-- adjClose: double (nullable = true)
 |-- Name: string (nullable = true)



Crear un vector en donde se encuentren todas las variables independientes

In [74]:
featassembler = VectorAssembler(inputCols=['adjClose'], outputCol = "Independent Features" )
featassembler

VectorAssembler_75b03536309c

In [75]:
result = featassembler.transform(df_spark_col)
result.show()

+------------------+---------+------------------+----+--------------------+
|              Open|   Volume|          adjClose|Name|Independent Features|
+------------------+---------+------------------+----+--------------------+
|17.959627151489258|2557142.0|  9.45626735687256| TSM|[17.9596271514892...|
| 18.24394989013672|2295467.0| 9.518234252929688| TSM|[18.2439498901367...|
| 17.77008056640625|1539456.0| 9.121644020080566| TSM|[17.7700805664062...|
| 18.05440139770508|1511230.0| 9.468661308288574| TSM|[18.0544013977050...|
| 18.69412422180176|1102626.0| 9.852862358093262| TSM|[18.6941242218017...|
| 18.81259155273437|2573761.0| 9.716533660888672| TSM|[18.8125915527343...|
|19.002140045166016|1407826.0|  9.87764835357666| TSM|[19.0021400451660...|
|18.954753875732425|1751803.0| 10.07594394683838| TSM|[18.9547538757324...|
| 19.52339553833008|4445851.0|10.943493843078612| TSM|[19.5233955383300...|
| 20.04465103149414|2009786.0|10.670833587646484| TSM|[20.0446510314941...|
|20.40005302

Columnas que se utilizarán en el modelo

Ya se juntarón las variables independientes en una sola columna, por ende las columnas en donde estan individuales ya no se tomaran en cuenta

In [76]:
final_data = result.select("Independent features", "adjClose")
final_data.show()

+--------------------+------------------+
|Independent features|          adjClose|
+--------------------+------------------+
|[17.9596271514892...|  9.45626735687256|
|[18.2439498901367...| 9.518234252929688|
|[17.7700805664062...| 9.121644020080566|
|[18.0544013977050...| 9.468661308288574|
|[18.6941242218017...| 9.852862358093262|
|[18.8125915527343...| 9.716533660888672|
|[19.0021400451660...|  9.87764835357666|
|[18.9547538757324...| 10.07594394683838|
|[19.5233955383300...|10.943493843078612|
|[20.0446510314941...|10.670833587646484|
|[20.4000530242919...|10.596468925476074|
|[20.6843738555908...|10.435358047485352|
|[20.9450016021728...|10.831949234008787|
|[21.3714847564697...|11.835826873779297|
|[22.6272354125976...| 12.98842716217041|
|[25.4941425323486...|12.542258262634276|
|[22.8878631591796...|11.674710273742676|
|[21.7979660034179...| 10.76998233795166|
|[20.1868114471435...|10.286632537841797|
|[19.9024906158447...|11.525986671447754|
+--------------------+------------

Creación de split de dataset para entrenamiento y pruebas

In [77]:
train_data, test_data = final_data.randomSplit([0.80, 0.20])

Creación y entrenamiento del modelo

In [78]:
from pyspark.ml.regression import LinearRegression

model = LinearRegression(featuresCol = 'Independent features', labelCol='adjClose')
model = model.fit(train_data)

Intercepto

In [79]:
model.intercept

44.593001058227316

Coeficientes

In [80]:
model.coefficients

DenseVector([0.9916, -0.0])

Predicción de los datos

In [81]:
prediction_result = model.evaluate(test_data)
prediction_result.predictions.show()

+--------------------+------------------+------------------+
|Independent features|          adjClose|        prediction|
+--------------------+------------------+------------------+
|[0.03439500182867...|0.0343950018286705|  44.6271080809619|
|[0.03439500182867...|0.0343950018286705|  44.6271080809619|
|[0.03439500182867...|0.0343950018286705|  44.6271080809619|
|[0.03439500182867...|0.0343950018286705|  44.6271080809619|
|[0.03439500182867...|0.0343950018286705|  44.6271080809619|
|[0.03439500182867...|0.0343950018286705|  44.6271080809619|
|[0.03439500182867...|0.0343950018286705|  44.6271080809619|
|[0.15500000119209...|0.2339999973773956| 44.65863015238019|
|[0.17000000178813...|0.1599999964237213|44.730163529803676|
|[0.17499999701976...|0.1749999970197677| 44.76646492010552|
|[0.17499999701976...|0.1749999970197677| 44.76369933846969|
|[0.18000000715255...|0.1899999976158142| 44.73298857314142|
|        [0.1875,0.0]|            0.1875|  44.7789311766378|
|[0.18999999761581...|0.

Evaluación del modelo

In [82]:
prediction_result.meanAbsoluteError, prediction_result.meanSquaredError

(1004.9540034617183, 4323608712.196476)

La evaluación del modelo tiene resultados muy malos, esto debido al tipo de predicción que se quiere hacer, debido a que hacer predicciones en econmia son muy complejas e inesactas, ya que si hubiese un modelo con gran exactitud se rompería la economia. Se podría mejorar el modelo por medio de la inserción de nuevas variables que tuvieran un mayor poder de explicación. Otra opción sería cambiar el tipo de modelo de uno lineal a un XGBoost